## Thông tin nhóm

**Thành viên thứ 1:**

- Họ và tên: Trần Trung Hiếu
- MSSV: 1712442

**Thành viên thứ 2:**
- Họ và tên: Lê Long Hồ
- MSSV: 1712447

**Thành viên thứ 3:**
- Họ và tên: Nguyễn Đình Thiên Phúc
- MSSV: 18120144

___

## Import

In [6]:
import csv
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
import re
pd.set_option('display.max_rows', 500)

---

## Thu thập dữ liệu

TODO

---

## Khám phá dữ liệu

In [2]:
movie_df = pd.read_csv('movie_final.csv')
movie_df.sample(5)

,url,title,score,posterImagePath,certification,overview,tagline,genres,trailerUrl,language
1483,https://www.themoviedb.org/tv/78508,Druck\n(2018),79.0,poster_final/78508.jpg,NaN,Skam Germany,NaN,Phim Chính Kịch,NaN,id
903,https://www.themoviedb.org/tv/433,Terminator: The Sarah Connor Chronicles\n(2008),74.0,poster_final/433.jpg,TV-14,The series picks up four years after the event...,NaN,"Sci-Fi & Fantasy, Phim Chính Kịch",NaN,en
4002,https://www.themoviedb.org/movie/781301,Homeschool Musical Class Of 2020\n(2020),0.0,poster_final/781301.jpg,NaN,Exploring the themes of classic teen movies th...,NaN,"Phim Nhạc, Phim Tài Liệu",NaN,en
1019,https://www.themoviedb.org/tv/2407,Dark Angel\n(2000),75.0,poster_final/2407.jpg,TV-14,Super soldier Max Guevera tries to live a norm...,NaN,"Action & Adventure, Phim Chính Kịch",NaN,en
2700,https://www.themoviedb.org/movie/768509,Captain Underpants: Mega Blissmas\n(2020),58.0,poster_final/768509.jpg,0,Christmas gets weird - really weird - after Ge...,NaN,"Phim Hoạt Hình, Phim Gia Đình, Phim Hài",NaN,en


In [10]:
movie_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5598 entries, 0 to 5597
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   url              5598 non-null   object 
 1   title            5594 non-null   object 
 2   score            5594 non-null   float64
 3   posterImagePath  5326 non-null   object 
 4   certification    4177 non-null   object 
 5   overview         5594 non-null   object 
 6   tagline          2522 non-null   object 
 7   genres           5230 non-null   object 
 8   trailerUrl       79 non-null     object 
 9   language         5594 non-null   object 
dtypes: float64(1), object(9)
memory usage: 437.5+ KB


In [3]:
n_row, n_col = movie_df.shape
print("Dữ liệu có số dòng:", n_row)
print("Dữ liệu có số cột:", n_col)

Dữ liệu có số dòng: 5598
Dữ liệu có số cột: 10


Mỗi dòng trong tập dữ liệu thể hiện vài thông tin của một bộ phim, bộ phim đó có thể là phim truyền hình hoặc phim điện ảnh. Các dòng này đều có ý nghĩa tương đương nhau. 

## Đưa ra câu hỏi cần trả lời

Các thông tin được tải lên trên Internet hầu như không được kiểm duyệt, chính vì vậy sẽ có thể xuất hiện những nội dung không phù hợp với mọi lứa tuổi, cụ thể là những bộ phim có gán nhãn giới hạn độ tuổi người lớn không nên để những người chưa đủ tuổi, đặc biệt là trẻ nhỏ tiếp xúc. Tuy nhiên, rất nhiều các website xem phim lại không có gán nhãn giới hạn độ tuổi. Chính vì vậy, câu hỏi này được đưa ra nhằm tìm câu trả lời giải quyết vấn đề nội dung bộ phim không phù hợp với các lứa tuổi, giúp các phụ huynh có thể quản lí được trẻ nhỏ không tìm coi những bộ phim có nội dung không phù hợp.

Câu hỏi này sẽ liên quan tới việc phân loại nội dung tóm tắt của một bộ phim và dự đoán độ tuổi phù hợp để coi bộ phim này là gì.

- Input: overview (nội dung tóm tắt của bộ phim)
- Output: certification (độ tuổi phù hợp)

---

## Khám phá dữ liệu

In [12]:
print("Danh sách các ngôn ngữ thể hiện nội dung tóm tắt:")
print(movie_df.language.value_counts())

Danh sách các ngôn ngữ thể hiện nội dung tóm tắt:
en    4304
vi    1224
lt      31
fi      10
tl       5
pt       4
cy       4
ro       3
es       2
de       1
af       1
it       1
fr       1
sk       1
cs       1
id       1
Name: language, dtype: int64


In [8]:
print('Kiểu dữ liệu của cột cerfitication:', movie_df.certification.dtype)

Kiểu dữ liệu của cột cerfitication: object


In [11]:
print("Số lượng dòng bị thiếu output:", movie_df.certification.isnull().sum())
print("Số lượng dòng có output:", movie_df.shape[0] - movie_df.certification.isnull().sum())

Số lượng dòng bị thiếu output: 1421
Số lượng dòng có output: 4177


In [25]:
print('Số lượng lớp trong certification:', len(movie_df.certification.unique()))
print('Tỉ lệ bộ phim của từng lớp:\n', movie_df.certification.value_counts(normalize=True)*100)

Số lượng lớp trong certification: 88
Tỉ lệ bộ phim của từng lớp:
 R             14.891070
TV-14         12.401245
PG-13         10.892985
PG             9.743835
TV-MA          9.743835
TV-PG          4.907829
G              4.141729
12             4.045966
NR             3.998085
15             3.303807
16             2.657410
TV-Y7          2.250419
TV-G           1.723725
U              1.484319
18             1.173091
14             1.125209
L              1.053388
16+            0.694278
0              0.598516
10             0.598516
6              0.550634
18+            0.478813
T              0.430931
12+            0.406991
MA15+          0.383050
TV-Y           0.359109
保護級            0.359109
普遍級            0.335169
AL             0.335169
6+             0.287288
7              0.287288
輔12級           0.239406
APTA           0.215466
UA             0.215466
NC-17          0.215466
M              0.215466
11             0.191525
M/12           0.167584
C13            0.14364

## Tiền xử lí - tách các tập dữ liệu

`en` và `vi` là hai ngôn ngữ có số lượng bộ phim chiếm nhiều nhất, cho nên nhóm sẽ giữ lại những bộ phim thuộc một trong hai ngôn ngữ này để thực hiện tiền xử lí và loại bỏ đi các dòng chứa các ngôn ngữ còn lại

In [40]:
envi_movie_df = movie_df[((movie_df.language == 'vi') | (movie_df.language == 'en'))]
envi_movie_df.reset_index(inplace=True)
envi_movie_df.head(5)

,index,url,title,score,posterImagePath,certification,overview,tagline,genres,trailerUrl,language
0,0,https://www.themoviedb.org/tv/44217,Huyền Thoại Vikings\n(2013),79.0,poster_final/44217.jpg,14+,Người Viking đưa chúng ta đến thế giới tàn khố...,NaN,"Action & Adventure, Phim Chính Kịch",NaN,vi
1,1,https://www.themoviedb.org/tv/82856,The Mandalorian\n(2019),85.0,poster_final/82856.jpg,TV-14,"After the fall of the Galactic Empire, lawless...",Bounty hunting is a complicated profession.,"Sci-Fi & Fantasy, Action & Adventure",NaN,en
2,2,https://www.themoviedb.org/tv/77169,Cobra Kai\n(2018),81.0,poster_final/77169.jpg,TV-14,This Karate Kid sequel series picks up 30 year...,Cobra Kai never dies.,"Action & Adventure, Phim Chính Kịch",NaN,en
3,3,https://www.themoviedb.org/tv/75006,Học Viện Umbrella\n(2019),87.0,poster_final/75006.jpg,TV-MA,"Được đưa về quá khứ những năm 1960 tại Dallas,...",Super. Dysfunctional. Family.,"Action & Adventure, Sci-Fi & Fantasy, Phim Chí...",NaN,vi
4,4,https://www.themoviedb.org/tv/71712,Bác Sĩ Thiên Tài\n(2017),86.0,poster_final/71712.jpg,TV-14,Một bác sĩ trẻ mắc hội chứng bác học được nhận...,"His mind is a mystery, his methods are a miracle.",Phim Chính Kịch,NaN,vi


---

## Khám phá dữ liệu - tập huấn luyện

## Tiền xử lí

In [31]:
movie_df[movie_df.score == 0.0]

,url,title,score,posterImagePath,certification,overview,tagline,genres,trailerUrl,language
125,https://www.themoviedb.org/tv/35674,Football Tonight\n(1998),0.0,NaN,NaN,We don't have an overview translated in Englis...,NaN,NaN,NaN,en
181,https://www.themoviedb.org/tv/9890,The Dick Cavett Show\n(1968),0.0,poster_final/9890.jpg,NaN,The Dick Cavett Show has been the title of sev...,NaN,Talk,NaN,en
192,https://www.themoviedb.org/tv/82395,The Liar\n(2012),0.0,poster_final/82395.jpg,NaN,We don't have an overview translated in Englis...,NaN,Reality,NaN,en
223,https://www.themoviedb.org/tv/23915,Markus Lanz\n(2008),0.0,NaN,NaN,We don't have an overview translated in Englis...,NaN,Talk,NaN,en
252,https://www.themoviedb.org/tv/99053,"Médicos, línea de vida\n(2020)",0.0,poster_final/99053.jpg,NaN,We don't have an overview translated in Englis...,NaN,NaN,NaN,en
...,...,...,...,...,...,...,...,...,...,...
4610,https://www.themoviedb.org/movie/619585,Matinee\n(2019),0.0,poster_final/619585.jpg,NaN,Satoshi Makino (Masaharu Fukuyama) is a genius...,NaN,Phim Lãng Mạn,NaN,en
4612,https://www.themoviedb.org/movie/606915,Sunshine Family\n(2019),0.0,poster_final/606915.jpg,PG,A modern Filipino family in South Korea tries ...,NaN,Phim Chính Kịch,NaN,en
4613,https://www.themoviedb.org/movie/587610,The Oath\n(2020),0.0,poster_final/587610.jpg,12+,"An impressing war drama, a story of life and h...",NaN,"Phim Chính Kịch, Phim Chiến Tranh, Phim Lịch Sử",NaN,en
4616,https://www.themoviedb.org/movie/783153,Angkara Dendam\n(2021),0.0,poster_final/783153.jpg,NaN,We don't have an overview translated in Englis...,NaN,Phim Chính Kịch,NaN,en


## Tiền xử lí - tập validation

TODO

---

## Mô hình hóa dữ liệu

### Hyperparameter tuning

TODO

### Đánh giá mô hình

TODO

---

## Kết luận

TODO